# Part1: Sensor Uncertainty prediction

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Préparation des données


In [2]:
uncertainty = pd.read_csv('Uncertainties-_1_.csv')
radar_data = pd.read_csv('output-_4_ (3) (1).csv')

In [3]:
radar_data.head()

,time,range_radar,velocity_radar,AOA_radar
0,0.0,378.929664,13.561884,-2.672516
1,0.0,248.211704,18.178031,-2.669837
2,0.0,212.676491,16.456920,-2.652078
3,0.0,122.375529,6.621422,0.179927
4,0.0,319.027840,19.032024,-2.645719


In [21]:
df=radar_data.drop(['time'],axis='columns')
df

,range_radar,velocity_radar,AOA_radar
0,378.929664,13.561884,-2.672516
1,248.211704,18.178031,-2.669837
2,212.676491,16.456920,-2.652078
3,122.375529,6.621422,0.179927
4,319.027840,19.032024,-2.645719
...,...,...,...
2346,52.539348,4.823031,-1.837647
2347,239.133326,3.685453,-2.587122
2348,249.465877,1.056951,-2.584789
2349,122.629183,0.792700,0.099250


In [5]:
df1=pd.concat([df, uncertainty], axis=1)
df1.head()

,range_radar,velocity_radar,AOA_radar,uncertainties
0,378.929664,13.561884,-2.672516,4.217167
1,248.211704,18.178031,-2.669837,0.560986
2,212.676491,16.456920,-2.652078,4.159594
3,122.375529,6.621422,0.179927,2.445053
4,319.027840,19.032024,-2.645719,2.040474


In [6]:
x=df1.drop('uncertainties',axis='columns')
x.head()

,range_radar,velocity_radar,AOA_radar
0,378.929664,13.561884,-2.672516
1,248.211704,18.178031,-2.669837
2,212.676491,16.456920,-2.652078
3,122.375529,6.621422,0.179927
4,319.027840,19.032024,-2.645719


In [7]:
y=df1.uncertainties
y.head()

0    4.217167
1    0.560986
2    4.159594
3    2.445053
4    2.040474
Name: uncertainties, dtype: float64

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)
x_test

,range_radar,velocity_radar,AOA_radar
168,282.549902,30.051679,1.144871
729,391.807439,15.670305,-2.739052
1284,248.283054,1.058331,-2.583976
1555,52.539348,4.823031,-1.837647
1417,46.177913,13.702505,-0.474339
...,...,...,...
54,226.605033,16.258083,-2.658504
1882,237.978565,1.075295,-2.573948
2168,231.401948,1.078134,-2.572264
1733,298.849418,17.706522,-2.683026


# Model Training

In [43]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_predict

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_train[:len(y_pred)], y_pred)
print("Mean Squared Error:", mse)

# Define a list of sample sizes to test
sample_sizes = [1000, 1500, 2000, 2351]

# Variables to store the best performance values
best_sample_size = None
best_r2_score = -float('inf')

# Loop through each sample size and evaluate the model
for sample_size in sample_sizes:
    # Select a subset of the data with the current sample size
    X_train_subset = x_train[:sample_size]
    y_train_subset = y_train[:sample_size]
    
    # Create a Random Forest Regressor model
    random_forest = RandomForestRegressor(n_estimators=100)
    
    # Train the model
    random_forest.fit(X_train_subset, y_train_subset)

    # Make predictions on the validation set
    y_pred = cross_val_predict(random_forest, x_train, y_train, cv=5)
    
    # Calculate the R2 score
    r2 = r2_score(y_train, y_pred)
    
    # Print the performance metrics
    print("Sample Size:", sample_size)
    print("R2 Score:", r2)
    print("----------------------------")
    
    # Check if the current sample size has a better R2 score
    if r2 > best_r2_score:
        best_r2_score = r2
        best_sample_size = sample_size

# Print the best sample size and its corresponding R2 score
print("Best Sample Size:", best_sample_size)
print("Best R2 Score:", best_r2_score)

# Use the best sample size to train the BaggingRegressor
bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=50)
bagging.fit(x_train[:best_sample_size], y_train[:best_sample_size])

# Calculate the out-of-bag (OOB) score
oob_score = bagging.score(x_train, y_train)
print("Out-of-Bag Score:", oob_score)


Mean Squared Error: 1.6963509621518675
Sample Size: 1000
R2 Score: 0.5373951682381255
----------------------------
Sample Size: 1500
R2 Score: 0.5432667385656742
----------------------------
Sample Size: 2000
R2 Score: 0.5423642828878532
----------------------------
Sample Size: 2351
R2 Score: 0.5417327762449338
----------------------------
Best Sample Size: 1500
Best R2 Score: 0.5432667385656742
Out-of-Bag Score: 0.6833210742309626


In [22]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor


# Define a list of sample sizes to test
sample_sizes = [1000, 1500, 2000, 2351]
# Variables to store the best performance values
best_sample_size = None
best_r2_score = -float('inf')

# Loop through each sample size and evaluate the model
for sample_size in sample_sizes:
    # Select a subset of the data with the current sample size
    x_train_subset = x_train[:sample_size]
    y_train_subset = y_train[:sample_size]
# Creating a Random Forest Regressor model
random_forest = RandomForestRegressor(n_estimators=100)

# Creating a BaggingRegressor with the Random Forest Regressor as the base estimator
bagging = BaggingRegressor(base_estimator=random_forest, n_estimators=50, oob_score=True)

# Training the BaggingRegressor with 90% of the data (using bootstrap sampling)
bagging.fit(x_train, y_train)

# Evaluating the model's performance on out-of-bag samples
oob_score = bagging.oob_score_

# Printing the OOB score
print("Out-of-Bag Score:", oob_score)
# Use the best sample size to train the BaggingRegressor
bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=50, oob_score=True)
bagging.fit(x_train[:best_sample_size], y_train[:best_sample_size])

# Evaluate the model's performance on out-of-bag samples
oob_score = bagging.oob_score_

# Print the OOB score
print("Out-of-Bag Score:", oob_score)

Out-of-Bag Score: 0.5472983544713494
Out-of-Bag Score: 0.5499926671053301


In [17]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
# Create a Random Forest Regressor model
random_forest = RandomForestRegressor(n_estimators=100)
    
# Train the model
random_forest.fit(x_train, y_train)
    
# Make predictions on the validation set
y_pred = random_forest.predict(x_test)
    
# Calculate the R2 score
r2 = r2_score(y_test, y_pred)
    
# Print the performance metrics
    
print("R2 Score:", r2)
print("----------------------------")

R2 Score: 0.631422174971634
----------------------------


In [491]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

0.3627618347077516

In [492]:
from sklearn.neighbors import KNeighborsRegressor
model1=KNeighborsRegressor()
model1.fit(x_train,y_train)
model1.score(x_test,y_test)

0.5443799514998496

In [493]:
from sklearn.ensemble import GradientBoostingRegressor
model1=GradientBoostingRegressor()
model1.fit(x_train,y_train)
model1.score(x_test,y_test) 

0.5750266048992003

In [501]:
from sklearn.ensemble import RandomForestRegressor
model1=RandomForestRegressor()
model1.fit(x_train,y_train)
y_test.head()
model1.score(x_test,y_test) 

0.6431115725009391

In [502]:
y_test.head()

168     4.520722
729     2.388368
1284    0.536364
1555    1.688602
1417    2.757646
Name: uncertainties, dtype: float64

In [503]:
from sklearn.model_selection import cross_val_predict
import numpy as np
from sklearn.metrics import mean_squared_error
y_pred_cv = cross_val_predict(model1, x_train, y_train, cv=5)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_train, y_pred_cv)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.47181566162423944


# Part2: Detection of the contradiction between the radar_data and camera_data

In [440]:
import pandas as pd

In [505]:
radar_data = pd.read_csv('outputRadar2.csv')
camera_data = pd.read_csv('outputCamera1.csv')

In [506]:
merged_data= pd.concat([radar_data, camera_data], axis=1)
merged_data.head()
merged_data=merged_data.drop(['time'],axis='columns')
merged_data.head()

,range_radar,velocity_radar,AoA_radar,range_camera,velocity_camera,AoA_camera
0,378.929664,13.561884,-2.672516,378.929664,13.561884,-2.672516
1,248.211704,18.178031,-2.669837,248.211704,18.178031,-2.669837
2,212.676491,16.456920,-2.652078,212.676491,16.456920,-2.652078
3,122.375529,6.621422,0.179927,122.375529,6.621422,0.179927
4,319.027840,19.032024,-2.645719,319.027840,19.032024,-2.645719


In [507]:
threshold=0.1

In [508]:
merged_data['contradiction'] = (abs(merged_data[' velocity_radar'] - merged_data[' velocity_camera']) > threshold) | (abs(merged_data[' AoA_radar'] - merged_data[' AoA_camera']) > threshold) | (abs(merged_data[' range_radar'] - merged_data[' range_camera']) > threshold)


In [510]:
X = merged_data[[' velocity_radar', ' AoA_radar', ' range_radar', ' velocity_camera', ' AoA_camera', ' range_camera']]
y = merged_data['contradiction']
y[0:50]

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10     True
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24     True
25    False
26     True
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38     True
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47     True
48    False
49    False
Name: contradiction, dtype: bool

In [511]:
k=0
for i in y: 
    if (i == True):
     print("Switch to manual mode!")
     k=1;
     break

Switch to manual mode!
